# Retrieving conversations

In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm, notebook

### Getting original data

Function to retrieve inital scraped tweet data oranized by date

In [ ]:
def get_inital_data(date):
    print('Starting with date:', date, '\n')
    df = pd.read_csv('../Data/premier_league_' + date + '.csv')
    df = df[(df['Reply_count'] > 0) & (df['inReplyToTweetId'].isna())]
    return df

### Retrieving conversation info

Function to recursively retrieve info about tweets belonging to the same conversations (by conversation id)

In [ ]:
def retrieve_conversations(c, total=10):  
    smode = sntwitter.TwitterTweetScraperMode.RECURSE
    if c not in retrieved_conversations:
        #print(len(retrieved_conversations),'/',total)
        retrieved_conversations.append(c)
        try:
            for i,tweet in enumerate(sntwitter.TwitterTweetScraper(tweetId=c, mode=smode).get_items()):
                if tweet.lang=='en':
                    conversations[(c,i)] = {
                        'tweetId': tweet.id,
                        'content': tweet.content, 
                        'userId':  tweet.user.id
                    }

        except Exception:
            print("!!! Some Tweets were not retrieved for conversation", c, "!!!\n")
            incomplete_conv.append(c)

### Saving the results to file

Converting conversation dictionary to a dataframe and saving it to a csv file

In [ ]:
def df_to_file(date, conversations):
    df_conversations = pd.DataFrame.from_dict(conversations, orient='index', 
        columns=['tweetId', 'content', 'userId']).reset_index()
    df_conversations.rename(columns = {'level_0':'conversation_id', 'level_1':'conv_indx'}, inplace = True)

    # To file
    df_conversations.to_csv('../Data/conversations_'+date+'.csv', index=False)
    print('Finished with date ' + date + '\n\n\n')

In [ ]:
# Initialize variables that are used for retrieval
retrieved_conversations = []
conversations = {}
incomplete_conv = []
notebook.tqdm.pandas()


# Specify dates (for files) 
dates = ['2018-08-10','2018-08-11','2019-05-11','2019-05-12','2019-05-13'] #['2018-08-09'] 

for d in dates:
    data = get_inital_data(d)
    uniq_conv = len(data.conversationId.unique())
    print("There are", uniq_conv , "unique conversation Id's")
    print('Retrieving conversations:')
    data['conversationId'].progress_apply(retrieve_conversations)
    df_to_file(d, conversations)

In [ ]:
ic = pd.DataFrame({'incomplete': incomplete_conv})
ic.head()
ic.to_csv('../Data/incomplete.csv', index=False)